In [1]:
import requests
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    certifi-2018.11.29         |        py36_1000         145 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    ca-certificates-2018.11.29 |       ha4d7672_0         143 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0            conda-forge
    geopy:           

In [2]:
CLIENT_ID = 'QMW0ZEL5J22NLOR1CJTEPO0AY51SDFMUTWDHWWLJZYOCLSSB' # your Foursquare ID
CLIENT_SECRET = 'B0ZL1YBPH3DML5SAGYKDMEM5ZRBWNARUT2G3ES43M2RXLYOR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [3]:
address = 'Detroit, Michigan'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)
radius = 50000

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


42.3315509 -83.0466403


In [4]:

search_query = 'Chinese'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)

In [5]:
results = requests.get(url).json()
venues = results['response']['venues']
dataframe = json_normalize(venues)

In [6]:
# Remove Canadian Locations
dataframe = dataframe.loc[dataframe['location.country']=="United States"]

#Include only relevant columns
df = dataframe[['id','name', 'location.address','location.formattedAddress','location.lat', 'location.lng']]

#Rename columns
df.rename(columns = {'id':'ID','name':'Name','location.address':'Address','location.formattedAddress':'Full Address','location.lat':'Latitude','location.lng':'Longitude'},inplace=True)

#Remove NAN address entries
df['Address']=df['Address'].astype(str)
df = df.loc[df['Address']!="nan"]
final=df.reset_index(drop=True)

#Add Cuisine Category to the dataframe
final['Cuisine']='Chinese'

final

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Pop these, since the values are in `kwargs` under different names
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,ID,Name,Address,Full Address,Latitude,Longitude,Cuisine
0,4bc0b69a74a9a593da7ed0f6,#1 Chinese Food,4226 E 10 Mile Rd,"[4226 E 10 Mile Rd (at Ryan), Warren, MI 48091...",42.476100,-83.063699,Chinese
1,4bfd86548992a593fe3facb0,Dynasty Chinese Food,1115 W Warren Ave,"[1115 W Warren Ave, Detroit, MI 48201, United ...",42.352946,-83.074169,Chinese
2,4c54817f1b46c9b6a9fcb3ce,Dynasty Chinese Food,13340 E Jefferson Ave,"[13340 E Jefferson Ave, Detroit, MI 48215, Uni...",42.370664,-82.948158,Chinese
3,4b3eccadf964a52034a225e3,Shogun Japanese and Chinese Bistro,23195 Marter Rd,"[23195 Marter Rd (Jefferson Ave), Saint Clair ...",42.460127,-82.883193,Chinese
4,4bfdda37b68d0f47397ae957,No 1 Chinese food,2145 E 8 Mile Rd,"[2145 E 8 Mile Rd, Warren, MI 48091, United St...",42.447622,-83.080992,Chinese
5,4e5444f71838d8d1daceae08,Panda Chinese,18614 Fenkell St,"[18614 Fenkell St (Grand River), Detroit, MI 4...",42.402244,-83.224081,Chinese
6,4f8f2123e4b0db47032a288e,Wing's Chinese Restaurant,18700 Van Dyke St,"[18700 Van Dyke St, Detroit, MI 48234, United ...",42.431153,-83.023951,Chinese
7,4d0bf3ad376f6dcbec6ed0ec,Peking Chinese Food,14855 Gratiot Ave,"[14855 Gratiot Ave, Detroit, MI 48205, United ...",42.442199,-82.973470,Chinese
8,4e243a9018a88345f068453b,Chinese herbs cosmetics center,10334 W. Warren,"[10334 W. Warren, Detroit, MI 48126, United St...",42.344710,-83.161150,Chinese
9,4f3299d219836c91c7e7a98f,Dynasty Express Chinese Restaurant,20622 Joy Rd,"[20622 Joy Rd, Detroit, MI 48228, United States]",42.351245,-83.241544,Chinese


In [9]:
final.to_excel('Chinese-restaurants-excel.xlsx')